# Adult dataset

In [1]:
# Import required libraries

import pandas as pd
import numpy as np
 
# Import necessary modules

from sklearn.ensemble import VotingClassifier
from sklearn import model_selection

import pandasql as psql

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt

from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

# Ignore harmless warnings 

import warnings 
warnings.filterwarnings("ignore")

In [2]:
# Load the 'Adult' dataset

adultdata1 = pd.read_csv(r"D:\iiit notes\Programming\AI\Internship practice\52 season Multinomial Classifier 03 - Aug - 2021\adult_train.csv", header=0)
adultdata2 = pd.read_csv(r"D:\iiit notes\Programming\AI\Internship practice\52 season Multinomial Classifier 03 - Aug - 2021\adult_validate.csv", header=0)
adultdata1.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
# Identify the missing data 

adultdata1.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
Salary            0
dtype: int64

In [4]:
adultdata1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32560 entries, 0 to 32559
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32560 non-null  int64 
 1   workclass       32560 non-null  object
 2   fnlwgt          32560 non-null  int64 
 3   education       32560 non-null  object
 4   education-num   32560 non-null  int64 
 5   marital-status  32560 non-null  object
 6   occupation      32560 non-null  object
 7   relationship    32560 non-null  object
 8   race            32560 non-null  object
 9   sex             32560 non-null  object
 10  capital-gain    32560 non-null  int64 
 11  capital-loss    32560 non-null  int64 
 12  hours-per-week  32560 non-null  int64 
 13  native-country  32560 non-null  object
 14  Salary          32560 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [5]:
adultdata1['native-country'].value_counts()

 United-States                 29170
 Mexico                          643
 ?                               583
 Philippines                     198
 Germany                         137
 Canada                          121
 Puerto-Rico                     114
 El-Salvador                     106
 India                           100
 Cuba                             95
 England                          90
 Jamaica                          81
 South                            80
 China                            75
 Italy                            73
 Dominican-Republic               70
 Vietnam                          67
 Guatemala                        64
 Japan                            62
 Poland                           60
 Columbia                         59
 Taiwan                           51
 Haiti                            44
 Iran                             43
 Portugal                         37
 Nicaragua                        34
 Peru                             31
 

In [6]:
# Display the variables of each column of adultdata1

for col in adultdata1.columns:
    print(adultdata1[col].value_counts())

36    898
31    888
34    886
23    877
35    876
     ... 
83      6
85      3
88      3
86      1
87      1
Name: age, Length: 73, dtype: int64
 Private             22695
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64
164190    13
123011    13
203488    13
126675    12
148995    12
          ..
218551     1
201204     1
362999     1
162297     1
145522     1
Name: fnlwgt, Length: 21647, dtype: int64
 HS-grad         10501
 Some-college     7290
 Bachelors        5355
 Masters          1723
 Assoc-voc        1382
 11th             1175
 Assoc-acdm       1067
 10th              933
 7th-8th           646
 Prof-school       576
 9th               514
 12th              433
 Doctorate         413
 5th-6th           333
 1st-4th           168
 Preschool          51
Name: education, dtype: int6

In [7]:
# Train & Test file: Replace '?' with most frequent 'Private' variable.

adultdata1['workclass'] = adultdata1['workclass'].str.replace('?', 'Private')
adultdata1['workclass'] = adultdata1['workclass'].astype(str)

In [8]:
# Train & Test file: Replace '?' with most frequent 'Adm-clerical' variable

adultdata1['occupation'] = adultdata1['occupation'].str.replace('?', 'Adm-clerical')
adultdata1['occupation'] = adultdata1['occupation'].astype(str)

In [9]:
# Train & Test file: Replace '?' with most frequent 'United-States' variable

adultdata1['native-country'] = adultdata1['native-country'].str.replace('?', 'United-States')
adultdata1['native-country'] = adultdata1['native-country'].astype(str)

In [10]:
# Train & Test file: Replace target variable 'Salary', and convert the 'Salary' to integer value.

adultdata1['Salary'] = adultdata1['Salary'].str.replace('<=50K', '0')
adultdata1['Salary'] = adultdata1['Salary'].str.replace('>50K', '1')
adultdata1['Salary'] = adultdata1['Salary'].astype(int)

In [11]:
# Display the variables of each column of adultdata2

for col in adultdata2.columns:
    print(adultdata2[col].value_counts())

35    461
33    460
23    452
36    450
38    437
     ... 
82      3
88      3
85      2
89      2
87      2
Name: age, Length: 73, dtype: int64
 Private             11210
 Self-emp-not-inc     1321
 Local-gov            1043
 ?                     963
 State-gov             683
 Self-emp-inc          579
 Federal-gov           472
 Without-pay             7
 Never-worked            3
Name: workclass, dtype: int64
136986    9
190290    8
125892    8
203488    8
127651    8
         ..
284701    1
344094    1
75140     1
342049    1
145522    1
Name: fnlwgt, Length: 12787, dtype: int64
 HS-grad         5283
 Some-college    3587
 Bachelors       2670
 Masters          934
 Assoc-voc        679
 11th             637
 Assoc-acdm       534
 10th             456
 7th-8th          309
 Prof-school      258
 9th              242
 12th             224
 Doctorate        181
 5th-6th          176
 1st-4th           79
 Preschool         32
Name: education, dtype: int64
9     5283
10    3587
13 

In [12]:
# Validation File: Replace '?' with most frequent 'Private' variable.

adultdata2['workclass'] = adultdata2['workclass'].str.replace('?', 'Private')
adultdata2['workclass'] = adultdata2['workclass'].astype(str)

In [13]:
# Validation File: Replace '?' with most frequent 'Private' variable.

adultdata2['occupation'] = adultdata2['occupation'].str.replace('?', 'Adm-clerical')
adultdata2['occupation'] = adultdata2['occupation'].astype(str)

In [14]:
# Validation File: '?' with most frequent 'United-States' variable

adultdata2['native-country'] = adultdata2['native-country'].str.replace('?', 'United-States')
adultdata2['native-country'] = adultdata2['native-country'].astype(str)

In [15]:
# Validation File: Replace target variable 'Salary', and convert the 'Salary' to integer value.

adultdata2['Salary'] = adultdata2['Salary'].str.replace('<=50K.', '0')
adultdata2['Salary'] = adultdata2['Salary'].str.replace('>50K.', '1')
adultdata2['Salary'] = adultdata2['Salary'].astype(int)

In [16]:
# Display the 'adultdata1' dataset information

adultdata1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32560 entries, 0 to 32559
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32560 non-null  int64 
 1   workclass       32560 non-null  object
 2   fnlwgt          32560 non-null  int64 
 3   education       32560 non-null  object
 4   education-num   32560 non-null  int64 
 5   marital-status  32560 non-null  object
 6   occupation      32560 non-null  object
 7   relationship    32560 non-null  object
 8   race            32560 non-null  object
 9   sex             32560 non-null  object
 10  capital-gain    32560 non-null  int64 
 11  capital-loss    32560 non-null  int64 
 12  hours-per-week  32560 non-null  int64 
 13  native-country  32560 non-null  object
 14  Salary          32560 non-null  int32 
dtypes: int32(1), int64(6), object(8)
memory usage: 3.6+ MB


In [17]:
# Display the 'adultdata2' dataset information

adultdata2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16281 entries, 0 to 16280
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             16281 non-null  int64 
 1   workclass       16281 non-null  object
 2   fnlwgt          16281 non-null  int64 
 3   education       16281 non-null  object
 4   education-num   16281 non-null  int64 
 5   marital-status  16281 non-null  object
 6   occupation      16281 non-null  object
 7   relationship    16281 non-null  object
 8   race            16281 non-null  object
 9   sex             16281 non-null  object
 10  capital-gain    16281 non-null  int64 
 11  capital-loss    16281 non-null  int64 
 12  hours-per-week  16281 non-null  int64 
 13  native-country  16281 non-null  object
 14  Salary          16281 non-null  int32 
dtypes: int32(1), int64(6), object(8)
memory usage: 1.8+ MB


In [18]:
# Display All columns in adultdata1 dataset

adultdata1.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'Salary'],
      dtype='object')

In [21]:
# Create cols1 to convert the variables into dummy

cols1 = ['workclass', 'education','marital-status', 'occupation', 'relationship', 
         'race', 'sex', 'native-country'] 

In [22]:
# Create cols2 for Scaling 

cols2 = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

In [23]:
# Train & Test file: Identify the dependent and Target variables

IndepVar = []
for col in adultdata1.columns:
    if col != 'Salary':
        IndepVar.append(col)
        
TargetVar = 'Salary'

x = adultdata1[IndepVar]
y = adultdata1[TargetVar]

In [24]:
# Train & Test file: Create dummy variable for all category variables

x = pd.get_dummies(x,columns=cols1)
x.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# Split into train and test sets

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state = 42)

# Copy the test data to back-up file

x_test_F1 = x_test.copy()

In [26]:
x_train.shape

(24420, 104)

In [27]:
# Validation file: Identify the dependent and Target variables

IndepVar = []
for col in adultdata2.columns:
    if col != 'Salary':
        IndepVar.append(col)

TargetVar = 'Salary'

x1 = adultdata2[IndepVar]
y1 = adultdata2[TargetVar]

In [28]:
# Validation file: Create dummy variable for all category variables 

x1 = pd.get_dummies(x1, columns=cols1)
x1.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,25,226802,7,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,0,0,50,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,28,336951,12,0,0,40,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,44,160323,10,7688,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,18,103497,10,0,0,30,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [29]:
x1.shape

(16281, 104)

In [30]:
# Matching columns in both files

set(x.columns).intersection(set(x1.columns))

{'age',
 'capital-gain',
 'capital-loss',
 'education-num',
 'education_ 10th',
 'education_ 11th',
 'education_ 12th',
 'education_ 1st-4th',
 'education_ 5th-6th',
 'education_ 7th-8th',
 'education_ 9th',
 'education_ Assoc-acdm',
 'education_ Assoc-voc',
 'education_ Bachelors',
 'education_ Doctorate',
 'education_ HS-grad',
 'education_ Masters',
 'education_ Preschool',
 'education_ Prof-school',
 'education_ Some-college',
 'fnlwgt',
 'hours-per-week',
 'marital-status_ Divorced',
 'marital-status_ Married-AF-spouse',
 'marital-status_ Married-civ-spouse',
 'marital-status_ Married-spouse-absent',
 'marital-status_ Never-married',
 'marital-status_ Separated',
 'marital-status_ Widowed',
 'native-country_ Cambodia',
 'native-country_ Canada',
 'native-country_ China',
 'native-country_ Columbia',
 'native-country_ Cuba',
 'native-country_ Dominican-Republic',
 'native-country_ Ecuador',
 'native-country_ El-Salvador',
 'native-country_ England',
 'native-country_ France',
 'nat

In [31]:
# settings to display all columns

pd.set_option("display.max_columns", None)
#pd.set_option("max_rows", None)
# display the dataframe head

x.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,education_ 9th,education_ Assoc-acdm,education_ Assoc-voc,education_ Bachelors,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college,marital-status_ Divorced,marital-status_ Married-AF-spouse,marital-status_ Married-civ-spouse,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,occupation_ Adm-clerical,occupation_ Armed-Forces,occupation_ Craft-repair,occupation_ Exec-managerial,occupation_ Farming-fishing,occupation_ Handlers-cleaners,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Female,sex_ Male,native-country_ Cambodia,native-country_ Canada,native-country_ China,native-country_ Columbia,native-country_ Cuba,native-country_ Dominican-Republic,native-country_ Ecuador,native-country_ El-Salvador,native-country_ England,native-country_ France,native-country_ Germany,native-country_ Greece,native-country_ Guatemala,native-country_ Haiti,native-country_ Honduras,native-country_ Hong,native-country_ Hungary,native-country_ India,native-country_ Iran,native-country_ Ireland,native-country_ Italy,native-country_ Jamaica,native-country_ Japan,native-country_ Laos,native-country_ Mexico,native-country_ Nicaragua,native-country_ Outlying-US(Guam-USVI-etc),native-country_ Peru,native-country_ Philippines,native-country_ Poland,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,39,77516,13,2174,0,40,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,40,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,40,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,40,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
# settings to display all columns

pd.set_option("display.max_columns", None)
#pd.set_option("max_rows", None)
# Display the dataframe head

x1.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,education_ 9th,education_ Assoc-acdm,education_ Assoc-voc,education_ Bachelors,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college,marital-status_ Divorced,marital-status_ Married-AF-spouse,marital-status_ Married-civ-spouse,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,occupation_ Adm-clerical,occupation_ Armed-Forces,occupation_ Craft-repair,occupation_ Exec-managerial,occupation_ Farming-fishing,occupation_ Handlers-cleaners,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Female,sex_ Male,native-country_ Cambodia,native-country_ Canada,native-country_ China,native-country_ Columbia,native-country_ Cuba,native-country_ Dominican-Republic,native-country_ Ecuador,native-country_ El-Salvador,native-country_ England,native-country_ France,native-country_ Germany,native-country_ Greece,native-country_ Guatemala,native-country_ Haiti,native-country_ Honduras,native-country_ Hong,native-country_ Hungary,native-country_ India,native-country_ Iran,native-country_ Ireland,native-country_ Italy,native-country_ Jamaica,native-country_ Japan,native-country_ Laos,native-country_ Mexico,native-country_ Nicaragua,native-country_ Outlying-US(Guam-USVI-etc),native-country_ Peru,native-country_ Philippines,native-country_ Poland,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,25,226802,7,0,0,40,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,0,0,50,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,28,336951,12,0,0,40,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,44,160323,10,7688,0,40,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,18,103497,10,0,0,30,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [36]:
# Scaling the features by using MinMaxScaler

from sklearn.preprocessing import MinMaxScaler

mmscaler = MinMaxScaler(feature_range=(0, 1))

x_train[cols2] = mmscaler.fit_transform(x_train[cols2])
x_train = pd.DataFrame(x_train)

x_test[cols2] = mmscaler.fit_transform(x_test[cols2])
x_test = pd.DataFrame(x_test)

In [37]:
# Scaling the features by using MinMaxScaler

from sklearn.preprocessing import MinMaxScaler

mmscaler = MinMaxScaler(feature_range=(0, 1))

x1[cols2] = mmscaler.fit_transform(x1[cols2])
x1 = pd.DataFrame(x1)

#x_test[cols2] = mmscaler.fit_transform(x_test[cols2])
#x_test = pd.DataFrame(x_test)

# Bagging Classifier with default 'base_estimator'

In [40]:
# Training bagging classifier - BaggingClassifier class of 'sklearn.ensemble' packages to build bagging classifier model. 
# We set default base estimator and set 100 to the number of estimators, then train the model 
# with train data.  

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

modelBAG = BaggingClassifier(base_estimator=None, n_estimators=100, max_samples=1.0, max_features=1.0,
                             bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False,
                             n_jobs=None, random_state=None, verbose=0)

modelBAG.fit(x_train,y_train)

# Predict model with test data 

y_pred = modelBAG.predict(x_test)
y_pred_prob = modelBAG.predict_proba(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# Predicted values

predicted = y_pred

# Confusion matrix

matrix = confusion_matrix(actual, predicted, labels=[1, 0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual, predicted, labels=[1, 0]).reshape(-1)

print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3)
accuracy = round((tp+tn)/(tp+tn+fp+fn), 3)
balanced_accuracy = round((sensitivity+specificity)/2, 3)

precision = round(tp/(tp+fp), 3)
f1score = round((2*tp/(2*tp+fp+fn)), 3)

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp*tn) - (fp*fn)) / sqrt(x), 3)

print('Accuracy : ', round(accuracy*100, 3), '%')
print('Precision :', round(precision*100, 3),'%')
print('Recall :', round(sensitivity*100,3), '%')
print('F1 Score :', round(f1score,3))
print('Balanced Accuracy :', round(balanced_accuracy*100, 3),'%')
print('MCC :', MCC)

# Area under ROC curve

from sklearn.metrics import roc_curve, roc_auc_score

print('roc_auc_score:', round(roc_auc_score(y_test, y_pred), 3))

Confusion matrix : 
 [[1236  714]
 [ 459 5731]]
Outcome values : 
 1236 714 459 5731
Classification report : 
               precision    recall  f1-score   support

           1       0.73      0.63      0.68      1950
           0       0.89      0.93      0.91      6190

    accuracy                           0.86      8140
   macro avg       0.81      0.78      0.79      8140
weighted avg       0.85      0.86      0.85      8140

Accuracy :  85.6 %
Precision : 72.9 %
Recall : 63.4 %
F1 Score : 0.678
Balanced Accuracy : 78.0 %
MCC : 0.588
roc_auc_score: 0.78


# Predicting with Validation dataset

In [42]:
# Predict model with test data 

y_pred = modelBAG.predict(x1)
y_pred_prob = modelBAG.predict_proba(x1)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y1

# predicted values

predicted = y_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)

print('Accuracy :', round(accuracy*100, 3),'%')
print('Precision :', round(precision*100, 3),'%')
print('Recall :', round(sensitivity*100,3), '%')
print('F1 Score :', round(f1Score,3))
print('Balanced Accuracy :', round(balanced_accuracy*100, 3),'%')
print('MCC :', MCC)

# Area under ROC curve 

from sklearn.metrics import roc_curve, roc_auc_score

print('roc_auc_score:', round(roc_auc_score(y1, y_pred), 3))

Confusion matrix : 
 [[ 2160  1686]
 [  931 11504]]
Outcome values : 
 2160 1686 931 11504
Classification report : 
               precision    recall  f1-score   support

           1       0.70      0.56      0.62      3846
           0       0.87      0.93      0.90     12435

    accuracy                           0.84     16281
   macro avg       0.79      0.74      0.76     16281
weighted avg       0.83      0.84      0.83     16281

Accuracy : 83.9 %
Precision : 69.9 %
Recall : 56.2 %
F1 Score : 0.623
Balanced Accuracy : 74.4 %
MCC : 0.527
roc_auc_score: 0.743


# Bagging Classifier with Decision Tree

In [41]:
# Training bagging classifier - BaggingClassifier class of 'sklearn.ensemble' packages to build bagging classifier model. 
# We set 'DecisionTreeClassifier' base estimator and set 100 to the number of estimators, then train the model 
# with train data.  

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

modelDT = DecisionTreeClassifier(criterion="entropy")

modelBAG = BaggingClassifier(base_estimator=modelDT, n_estimators=100, max_samples=1.0, max_features=1.0,
                             bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False,
                             n_jobs=None, random_state=None, verbose=0)

modelBAG.fit(x_train,y_train)

# Predict model with test data 

y_pred = modelBAG.predict(x_test)
y_pred_prob = modelBAG.predict_proba(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)

print('Accuracy :', round(accuracy*100, 3),'%')
print('Precision :', round(precision*100, 3),'%')
print('Recall :', round(sensitivity*100,3), '%')
print('F1 Score :', round(f1Score,3))
print('Balanced Accuracy :', round(balanced_accuracy*100, 3),'%')
print('MCC :', MCC)

# Area under ROC curve 

from sklearn.metrics import roc_curve, roc_auc_score

print('roc_auc_score:', round(roc_auc_score(y_test, y_pred), 3))

Confusion matrix : 
 [[1235  715]
 [ 446 5744]]
Outcome values : 
 1235 715 446 5744
Classification report : 
               precision    recall  f1-score   support

           1       0.73      0.63      0.68      1950
           0       0.89      0.93      0.91      6190

    accuracy                           0.86      8140
   macro avg       0.81      0.78      0.79      8140
weighted avg       0.85      0.86      0.85      8140

Accuracy : 85.7 %
Precision : 73.5 %
Recall : 63.3 %
F1 Score : 0.68
Balanced Accuracy : 78.0 %
MCC : 0.592
roc_auc_score: 0.781


# Bagging Classifier with Logistic Regression

In [43]:
# Training bagging classifier - BaggingClassifier class of 'sklearn.ensemble' packages to build bagging classifier model. 
# We set 'LogisticRegression' base estimator and set 100 to the number of estimators, then train the model 
# with train data.  

from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression

modelLR = LogisticRegression()

modelBAG = BaggingClassifier(base_estimator=modelLR, n_estimators=100, max_samples=1.0, max_features=1.0,
                             bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False,
                             n_jobs=None, random_state=None, verbose=0)

modelBAG.fit(x_train,y_train)

# Predict model with test data 

y_pred = modelBAG.predict(x_test)
y_pred_prob = modelBAG.predict_proba(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)

print('Accuracy :', round(accuracy*100, 3),'%')
print('Precision :', round(precision*100, 3),'%')
print('Recall :', round(sensitivity*100,3), '%')
print('F1 Score :', round(f1Score,3))
print('Balanced Accuracy :', round(balanced_accuracy*100, 3),'%')
print('MCC :', MCC)

# Area under ROC curve 

from sklearn.metrics import roc_curve, roc_auc_score

print('roc_auc_score:', round(roc_auc_score(y_test, y_pred), 3))

Confusion matrix : 
 [[1183  767]
 [ 412 5778]]
Outcome values : 
 1183 767 412 5778
Classification report : 
               precision    recall  f1-score   support

           1       0.74      0.61      0.67      1950
           0       0.88      0.93      0.91      6190

    accuracy                           0.86      8140
   macro avg       0.81      0.77      0.79      8140
weighted avg       0.85      0.86      0.85      8140

Accuracy : 85.5 %
Precision : 74.2 %
Recall : 60.7 %
F1 Score : 0.667
Balanced Accuracy : 77.0 %
MCC : 0.581
roc_auc_score: 0.77


# Bagging Classifier with KNN

In [45]:
# Training bagging classifier - BaggingClassifier class of 'sklearn.ensemble' packages to build bagging classifier model. 
# We set 'KNeighborsClassifier' base estimator and set 100 to the number of estimators, then train the model 
# with train data.  

from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

modelKNN = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30,
                                p=2, metric='minkowski', metric_params=None, n_jobs=None)

modelBAG = BaggingClassifier(base_estimator=modelKNN, n_estimators=100, max_samples=1.0, max_features=1.0,
                             bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False,
                             n_jobs=None, random_state=None, verbose=0)

modelBAG.fit(x_train,y_train)

# Predict model with test data 

y_pred = modelBAG.predict(x_test)
y_pred_prob = modelBAG.predict_proba(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)

print('Accuracy :', round(accuracy*100, 3),'%')
print('Precision :', round(precision*100, 3),'%')
print('Recall :', round(sensitivity*100,3), '%')
print('F1 Score :', round(f1Score,3))
print('Balanced Accuracy :', round(balanced_accuracy*100, 3),'%')
print('MCC :', MCC)

# Area under ROC curve 

from sklearn.metrics import roc_curve, roc_auc_score

print('roc_auc_score:', round(roc_auc_score(y_test, y_pred), 3))

Confusion matrix : 
 [[1151  799]
 [ 568 5622]]
Outcome values : 
 1151 799 568 5622
Classification report : 
               precision    recall  f1-score   support

           1       0.67      0.59      0.63      1950
           0       0.88      0.91      0.89      6190

    accuracy                           0.83      8140
   macro avg       0.77      0.75      0.76      8140
weighted avg       0.83      0.83      0.83      8140

Accuracy : 83.2 %
Precision : 67.0 %
Recall : 59.0 %
F1 Score : 0.627
Balanced Accuracy : 74.9 %
MCC : 0.521
roc_auc_score: 0.749


# Bagging Classifier with Random Forest

In [44]:
# Training bagging classifier - BaggingClassifier class of 'sklearn.ensemble' packages to build bagging classifier model. 
# We set 'RandomForestClassifier' base estimator and set 100 to the number of estimators, then train the model 
# with train data.  

from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

modelRF = RandomForestClassifier(criterion='gini', n_estimators=100, random_state=0)

modelBAG = BaggingClassifier(base_estimator=modelRF, n_estimators=100, max_samples=1.0, max_features=1.0,
                             bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False,
                             n_jobs=None, random_state=None, verbose=0)

modelBAG.fit(x_train,y_train)

# Predict model with test data 

y_pred = modelBAG.predict(x_test)
y_pred_prob = modelBAG.predict_proba(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)

print('Accuracy :', round(accuracy*100, 3),'%')
print('Precision :', round(precision*100, 3),'%')
print('Recall :', round(sensitivity*100,3), '%')
print('F1 Score :', round(f1Score,3))
print('Balanced Accuracy :', round(balanced_accuracy*100, 3),'%')
print('MCC :', MCC)

# Area under ROC curve 

from sklearn.metrics import roc_curve, roc_auc_score

print('roc_auc_score:', round(roc_auc_score(y_test, y_pred), 3))

Confusion matrix : 
 [[1249  701]
 [ 407 5783]]
Outcome values : 
 1249 701 407 5783
Classification report : 
               precision    recall  f1-score   support

           1       0.75      0.64      0.69      1950
           0       0.89      0.93      0.91      6190

    accuracy                           0.86      8140
   macro avg       0.82      0.79      0.80      8140
weighted avg       0.86      0.86      0.86      8140

Accuracy : 86.4 %
Precision : 75.4 %
Recall : 64.1 %
F1 Score : 0.693
Balanced Accuracy : 78.8 %
MCC : 0.609
roc_auc_score: 0.787


# Bagging Classifier with SVM

In [46]:
# Training bagging classifier - BaggingClassifier class of 'sklearn.ensemble' packages to build bagging classifier model. 
# We set 'SVC' base estimator and set 100 to the number of estimators, then train the model 
# with train data.  

from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC

modelSVMGaussian = SVC(kernel='rbf', random_state = None, class_weight=None,probability=True)

modelBAG = BaggingClassifier(base_estimator=modelSVMGaussian, n_estimators=100, max_samples=1.0, max_features=1.0,
                             bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False,
                             n_jobs=None, random_state=None, verbose=0)

modelBAG.fit(x_train,y_train)

# Predict model with test data 

y_pred = modelBAG.predict(x_test)
y_pred_prob = modelBAG.predict_proba(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)

print('Accuracy :', round(accuracy*100, 3),'%')
print('Precision :', round(precision*100, 3),'%')
print('Recall :', round(sensitivity*100,3), '%')
print('F1 Score :', round(f1Score,3))
print('Balanced Accuracy :', round(balanced_accuracy*100, 3),'%')
print('MCC :', MCC)

# Area under ROC curve 

from sklearn.metrics import roc_curve, roc_auc_score

print('roc_auc_score:', round(roc_auc_score(y_test, y_pred), 3))

Confusion matrix : 
 [[1016  934]
 [ 384 5806]]
Outcome values : 
 1016 934 384 5806
Classification report : 
               precision    recall  f1-score   support

           1       0.73      0.52      0.61      1950
           0       0.86      0.94      0.90      6190

    accuracy                           0.84      8140
   macro avg       0.79      0.73      0.75      8140
weighted avg       0.83      0.84      0.83      8140

Accuracy : 83.8 %
Precision : 72.6 %
Recall : 52.1 %
F1 Score : 0.607
Balanced Accuracy : 73.0 %
MCC : 0.519
roc_auc_score: 0.729
